In [5]:
import pygame
import random
import time
from functools import  lru_cache

# Pygame setup
pygame.init()
WIDTH, HEIGHT = 3200, 300
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Task Scheduler Visualization")
FONT = pygame.font.SysFont("Arial", 24)


def generate_task(qty: int):
    ids = list(range(1, qty))
    random.shuffle(ids)
    return [(i, random.randint(1, 10), random.randint(1, 10), random.randint(1, 10)) for i in ids]
# Task definition: [Task ID, Duration, Deadline, Priority]
# TASKS = [
#     (1, 2, 5, 10),
#     (2, 1, 3, 5),
#     (3, 2, 6, 8),
#     (4, 3, 7, 7),
#     (5, 1, 4, 6),
#     (6, 2, 5, 9),
#     (7, 1, 3, 4),
#     (8, 3, 7, 6),
#     (9, 2, 6, 8),
#     (10, 1, 4, 5),
#     (11, 2, 5, 7),
#     (12, 1, 3, 3),
#     (13, 3, 7, 9),
#     (14, 2, 6, 10),
#     (15, 1, 4, 6),
#     (16, 3, 7, 8),
#     (17, 2, 5, 9),
#     (18, 1, 3, 4),
#     (19, 3, 7, 7),
#     (20, 2, 6, 8),
# ]

TASK = generate_task(20)

GENERATIONS = 500
MUTATION_RATE = 0.1
POPULATION_SIZE = 100

def fitness(schedule):
    current_time = 0
    total_fitness = 0
    
    for task_id in schedule:
        # Get task properties
        duration, deadline, priority = next((t[1:] for t in TASKS if t[0] == task_id), (0, 0, 0))
        current_time += duration
        
        # Base reward for completing the task
        total_fitness += priority * 5  # Increase the weight of priority
        
        # Deadline handling
        if current_time <= deadline:
            # Extra bonus for early completion
            total_fitness += (deadline - current_time) * 2
        else:
            # Penalty for missing the deadline
            delay = current_time - deadline
            total_fitness -= delay * 1  # Reduce penalty impact

    return max(total_fitness, 0)

# Fitness function
def _2fitness(schedule):
    current_time = 0
    total_fitness = 0
    
    for task_id in schedule:
        duration, deadline, priority = next((t[1:] for t in TASKS if t[0] == task_id), (0, 0, 0))
        current_time += duration
        if current_time <= deadline:
            penalty = (current_time - deadline) / deadline  # Normalized delay
            penalty *= 0.5 * priority  # Reduce penalty impact
            total_fitness -= penalty

    return total_fitness

def _fitness(schedule):
    current_time = 0
    total_fitness = 0
    
    for task_id in schedule:
        duration, deadline, priority = next((t[1:] for t in TASKS if t[0] == task_id), (0, 0, 0))
        current_time += duration
        if current_time <= deadline:
            total_fitness += priority
        else:
            total_fitness -= priority # (current_time - deadline) # Penalize missed deadlines
    return total_fitness

# Generate a random individual (schedule)
def create_individual():
    task_ids = [task[0] for task in TASKS]
    random.shuffle(task_ids)
    return task_ids

# Generate initial population
def create_population(size):
    return [create_individual() for _ in range(size)]

# Selection: Choose top individuals based on fitness
def select(population):
    population.sort(key=fitness)
    # Keep top 2 individuals (elitism)
    elites = population[:2]
    # Select the rest
    selected = random.choices(population[:len(population) // 2], k=POPULATION_SIZE - 2)
    return elites + selected

# Crossover: Combine two parents to produce offspring
def crossover(parent1, parent2):
    split = len(parent1) // 2
    child = parent1[:split] + [task for task in parent2 if task not in parent1[:split]]
    return child

# Mutation: Randomly swap two tasks in the schedule
def mutate(individual):
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

@lru_cache(maxsize=None)
def create_color(task_id: str):
    return (random.randint(20, 255), random.randint(30, 255), random.randint(40, 255))

# Visualization of the schedule
def draw_schedule(schedule, generation, best_fitness):
    SCREEN.fill((255, 255, 255))  # Clear screen with white background

    # Draw tasks as rectangles on the timeline
    current_time = 0
    for task_id in schedule:
        duration, _, _ = next((t[1:] for t in TASKS if t[0] == task_id), (0, 0, 0))
        x = current_time * 100  # Scale time by 100 pixels
        y = HEIGHT // 2 - 20
        width = duration * 100  # Task duration width
        task_id = f"T{task_id}"
        pygame.draw.rect(SCREEN, create_color(task_id), (x, y, width, 40), border_radius=5)
        task_text = FONT.render(task_id, True, (255, 255, 255))
        SCREEN.blit(task_text, (x + 10, y + 10))
        current_time += duration

    # Draw time axis
    pygame.draw.line(SCREEN, (0, 0, 0), (0, HEIGHT // 2 + 30), (WIDTH, HEIGHT // 2 + 30), 2)

    # Display generation and best fitness
    gen_text = FONT.render(f"Generation: {generation}", True, (0, 0, 0))
    fit_text = FONT.render(f"Best Fitness: {best_fitness}", True, (0, 0, 0))
    SCREEN.blit(gen_text, (10, 10))
    SCREEN.blit(fit_text, (10, 70))

    pygame.display.flip()  # Update the display


# Genetic Algorithm with Visualization
def genetic_algorithm():
    population = create_population(POPULATION_SIZE)
    best_schedule = None

    for generation in range(GENERATIONS):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        # Selection
        selected = select(population)

        # Crossover and Mutation
        next_generation = []
        while len(next_generation) < POPULATION_SIZE:
            parent1, parent2 = random.sample(selected, 2)
            offspring = crossover(parent1, parent2)
            offspring = mutate(offspring)
            next_generation.append(offspring)

        population = next_generation
        best_schedule = max(population, key=fitness)
        best_fitness = fitness(best_schedule)
        print(f"Generation: {generation + 1}, Best Fitness: {best_fitness} - {best_schedule}")


        # Draw the schedule
        draw_schedule(best_schedule, generation + 1, best_fitness)

        # Pause briefly to simulate real-time update
        time.sleep(0.1)

    return (best_schedule, fitness(best_schedule))

# Run the Genetic Algorithm
best_schedule = genetic_algorithm()
print(f"Best schedule: {best_schedule[0]}, Fitness: {best_schedule[1]}")
pygame.quit()

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default


Generation: 1, Best Fitness: 0 - [12, 7, 18, 17, 3, 14, 6, 20, 16, 1, 11, 4, 19, 13, 15, 9, 5, 2, 10, 8]
Generation: 2, Best Fitness: 0 - [4, 1, 10, 15, 17, 8, 5, 6, 20, 13, 16, 7, 9, 2, 12, 11, 18, 14, 19, 3]
Generation: 3, Best Fitness: 0 - [14, 15, 11, 8, 9, 10, 3, 2, 20, 1, 12, 19, 17, 16, 4, 5, 6, 13, 18, 7]
Generation: 4, Best Fitness: 0 - [14, 15, 11, 8, 9, 10, 3, 2, 20, 1, 18, 16, 17, 5, 7, 6, 12, 4, 13, 19]
Generation: 5, Best Fitness: 0 - [17, 6, 2, 20, 18, 12, 19, 13, 5, 16, 8, 4, 7, 9, 11, 10, 14, 1, 15, 3]
Generation: 6, Best Fitness: 0 - [16, 8, 5, 7, 9, 6, 2, 12, 11, 10, 14, 1, 13, 20, 15, 3, 19, 17, 4, 18]
Generation: 7, Best Fitness: 0 - [6, 14, 9, 12, 1, 8, 16, 2, 20, 15, 5, 17, 3, 7, 4, 13, 11, 10, 18, 19]
Generation: 8, Best Fitness: 0 - [16, 8, 4, 7, 9, 6, 2, 12, 11, 10, 14, 17, 3, 15, 13, 1, 5, 18, 20, 19]
Generation: 9, Best Fitness: 0 - [1, 5, 14, 17, 18, 13, 7, 12, 8, 20, 6, 9, 16, 2, 15, 4, 11, 10, 3, 19]
Generation: 10, Best Fitness: 0 - [10, 14, 9, 12, 1, 8,

In [6]:
import pygame
import random
import time
from functools import lru_cache

# Pygame setup
pygame.init()
WIDTH, HEIGHT = 3200, 300  # Adjusted for better visualization
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Task Scheduler Visualization")
FONT = pygame.font.SysFont("Arial", 18)

# Generate tasks
def generate_tasks(qty):
    tasks = []
    for i in range(1, qty + 1):
        duration = random.randint(2, 10)
        deadline = random.randint(duration, duration + 10)
        priority = random.randint(1, 10)
        tasks.append((i, duration, deadline, priority))
    return tasks

# Task definition: [Task ID, Duration, Deadline, Priority]
TASKS = generate_tasks(20)

GENERATIONS = 200
POPULATION_SIZE = 10
MUTATION_RATE = 0.2

def fitness(schedule):
    current_time = 0
    total_penalty = 0

    for task_id in schedule:
        # Get task properties
        duration, deadline, priority = next((t[1:] for t in TASKS if t[0] == task_id), (0, 0, 0))
        current_time += duration

        # Calculate penalty for missing deadline
        if current_time > deadline:
            delay = current_time - deadline
            # Penalty is weighted by priority and delay
            total_penalty += delay * priority

    # Lower penalty is better
    return total_penalty

# Generate a random individual (schedule)
def create_individual():
    task_ids = [task[0] for task in TASKS]
    random.shuffle(task_ids)
    return task_ids

# Generate initial population
def create_population(size):
    return [create_individual() for _ in range(size)]

# Selection: Choose individuals based on fitness (lower is better)
def select(population):
    population.sort(key=fitness)
    # Keep top 2 individuals (elitism)
    elites = population[:2]
    # Select the rest using tournament selection
    selected = []
    for _ in range(POPULATION_SIZE - 2):
        tournament = random.sample(population[:POPULATION_SIZE // 2], 5)
        winner = min(tournament, key=fitness)
        selected.append(winner)
    return elites + selected

# Crossover: Order Crossover (OX)
def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None]*size
    # Copy a slice from parent1 to child
    child[start:end+1] = parent1[start:end+1]
    # Fill the remaining positions with genes from parent2
    p2_index = 0
    for i in range(size):
        if child[i] is None:
            while parent2[p2_index] in child:
                p2_index += 1
            child[i] = parent2[p2_index]
            p2_index += 1
    return child

# Mutation: Swap Mutation
def mutate(individual):
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

@lru_cache(maxsize=None)
def create_color(task_id):
    random.seed(task_id)
    return (random.randint(50, 200), random.randint(50, 200), random.randint(50, 200))

# Visualization of the schedule
def draw_schedule(schedule, generation, best_fitness):
    SCREEN.fill((255, 255, 255))  # Clear screen with white background

    # Draw tasks as rectangles on the timeline
    current_time = 0
    max_time = sum([t[1] for t in TASKS])
    time_scale = WIDTH / max_time

    for task_id in schedule:
        duration, deadline, priority = next((t[1:] for t in TASKS if t[0] == task_id), (0, 0, 0))
        x = current_time * time_scale
        y = HEIGHT // 2 - 20
        width = duration * time_scale
        color = create_color(task_id)
        pygame.draw.rect(SCREEN, color, (x, y, width, 40), border_radius=5)
        task_label = FONT.render(f"T{task_id}", True, (255, 255, 255))
        SCREEN.blit(task_label, (x + 5, y + 5))
        current_time += duration

    # Draw time axis
    pygame.draw.line(SCREEN, (0, 0, 0), (0, HEIGHT // 2 + 30), (WIDTH, HEIGHT // 2 + 30), 2)

    # Display generation and best fitness
    gen_text = FONT.render(f"Generation: {generation}", True, (0, 0, 0))
    fit_text = FONT.render(f"Total Penalty: {best_fitness}", True, (0, 0, 0))
    SCREEN.blit(gen_text, (10, 10))
    SCREEN.blit(fit_text, (10, 30))

    pygame.display.update()

# Genetic Algorithm with Visualization
def genetic_algorithm():
    population = create_population(POPULATION_SIZE)
    best_schedule = None

    for generation in range(1, GENERATIONS + 1):
        # Selection
        selected = select(population)

        # Crossover and Mutation
        next_generation = []
        while len(next_generation) < POPULATION_SIZE:
            parent1, parent2 = random.sample(selected, 2)
            child = crossover(parent1, parent2)
            child = mutate(child)
            next_generation.append(child)

        population = next_generation
        best_schedule = min(population, key=fitness)
        best_fitness = fitness(best_schedule)
        print(f"Generation: {generation}, Total Penalty: {best_fitness} - {best_schedule}")

        # Draw the schedule
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        draw_schedule(best_schedule, generation, best_fitness)
        pygame.time.delay(100)

    pygame.quit()
    return best_schedule, best_fitness

# Run the Genetic Algorithm
best_schedule, best_fitness = genetic_algorithm()
print(f"Best schedule: {best_schedule}, Total Penalty: {best_fitness}")

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default


Generation: 1, Total Penalty: 4420 - [14, 7, 11, 16, 8, 2, 10, 17, 13, 18, 3, 12, 4, 15, 1, 6, 20, 9, 5, 19]
Generation: 2, Total Penalty: 4412 - [3, 12, 4, 16, 8, 2, 10, 17, 13, 18, 11, 15, 14, 7, 1, 6, 20, 9, 5, 19]
Generation: 3, Total Penalty: 4412 - [3, 12, 4, 16, 8, 2, 10, 17, 13, 18, 11, 15, 14, 7, 1, 6, 20, 9, 5, 19]
Generation: 4, Total Penalty: 3911 - [3, 12, 9, 16, 8, 2, 10, 17, 13, 18, 11, 15, 14, 7, 1, 6, 20, 4, 5, 19]
Generation: 5, Total Penalty: 3792 - [3, 12, 9, 16, 8, 2, 5, 17, 13, 18, 11, 15, 14, 7, 1, 6, 20, 4, 10, 19]
Generation: 6, Total Penalty: 3792 - [3, 12, 9, 16, 8, 2, 5, 17, 13, 18, 11, 15, 14, 7, 1, 6, 20, 4, 10, 19]
Generation: 7, Total Penalty: 3789 - [3, 12, 9, 19, 8, 2, 5, 17, 13, 18, 11, 15, 14, 7, 1, 6, 20, 4, 10, 16]
Generation: 8, Total Penalty: 3669 - [3, 12, 9, 6, 8, 2, 5, 17, 13, 18, 11, 15, 14, 7, 1, 19, 20, 4, 10, 16]
Generation: 9, Total Penalty: 3486 - [3, 12, 9, 6, 8, 2, 16, 17, 13, 18, 11, 15, 14, 7, 1, 19, 20, 4, 10, 5]
Generation: 10, Tot